In [1]:
import numpy as np
source1 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-pca.txt"
source2 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-pca-cls.txt"
source3 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-svd.txt"
source4 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-svd-cls.txt"
source5 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-c5.txt"
source6 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-c10.txt"
source7 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-d5.txt"
source8 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-d10.txt"
source9 = "F:/MSc DS/Project/dms/trained_dms_txt/word2dm.txt"



def get_dict_from_source(source = source5):
    dim =17
    if source ==source1 or source==source3:
        dim = 15
    whole =[]

    with open(source) as f:
        for line in f:
            linelist = line.split()
            for a in linelist:
                whole.append(a)
    chunk = dim**2+1
    n = len(whole)/chunk
    voc = []
    for i in range(int(n)):
        voc.append(whole[i * chunk])
    dic = {}
    for i in range(int(n)):
        voc.append(whole[i * chunk])
    dic = {}
    for i in range(int(n)):
        a = whole[i * chunk + 1:(i + 1) * chunk]
        a = np.array(a, dtype='float')
        a.resize([dim, dim])
        value,_ = np.linalg.eig(a)
        maxeig = max(value)
        dic[voc[i]] = a/maxeig

    return dic


def kba(a, b):
    m = b-a
    eig,_ = np.linalg.eig(m)
    Kba = sum(eig)/sum(abs(eig))
    return Kba

def halfmatrix(m):
    eig,vec = np.linalg.eig(m)
    n = len(eig)
    for i in range(n):
        if eig[i]<0:
            eig[i]=0
    eignew = eig**0.5
    dim = m.shape[0]
    e = np.eye(dim)
    for i in range(dim):
        e[i][i] =eignew[i]
    vec_T = np.linalg.inv(vec)
    half = vec@e@vec_T
    return half

def compute_Kba(listA,listB, dic, method ='add'):
    '''
    imput two phrase or sentences lists
    tokenize each pair and get new dm using composition method
    compute Ke pred for each pair
    if oov set Ke to oov  # after test all in voc

    composition method
    add or pointwise
    '''

    n = len(listA)
    if method == 'add':
        Kba_list = []
        for i in range(n):
            phraseA = listA[i].split()
            phraseB = listB[i].split()
            newA = dic[phraseA[0].lower()] + dic[phraseA[1].lower()]
            newB = dic[phraseB[0].lower()] + dic[phraseB[1].lower()]
            Kba_list.append(kba(newA, newB))

    if method == 'pointwise':
        Kba_list = []
        for i in range(n):
            phraseA = listA[i].split()
            phraseB = listB[i].split()
            newA = dic[phraseA[0].lower()] * dic[phraseA[1].lower()]
            newB = dic[phraseB[0].lower()] * dic[phraseB[1].lower()]
            Kba_list.append(kba(newA, newB))
            
#     if method == 'dot':
#         Kba_list = []
#         for i in range(n):
#             phraseA = listA[i].split()
#             phraseB = listB[i].split()
#             newA = dic[phraseA[0].lower()]@dic[phraseA[1].lower()]
#             newB = dic[phraseB[0].lower()]@dic[phraseB[1].lower()]
#             Kba_list.append(kba(newA, newB))
    
    if method == 'mult':
        Kba_list = []
        for i in range(n):
            phraseA = listA[i].split()
            phraseB = listB[i].split()
            v1 = dic[phraseA[0].lower()]
            n1 = dic[phraseA[1].lower()]
            v2 = dic[phraseB[0].lower()]
            n2 = dic[phraseB[1].lower()]
            newA = halfmatrix(n1)*v1*halfmatrix(n1)
            newB = halfmatrix(n2)*v2*halfmatrix(n2)
            Kba_list.append(kba(newA, newB))
            
    return Kba_list


# get sv dataset
dicA = []
dicB = []
val = []
with open("F:/MSc DS/Project/compositional datasets/KS2016-VO.txt") as f:
    for line in f:
        linelist = line.split(",")
        dicA.append(linelist[0])
        dicB.append(linelist[1])
        val.append(linelist[2])

# convert the T to 1 F to 0 in val list
n = len(val)
for i in range(n):
    if val[i]=='T\n':
        val[i]=1
    else:
        val[i]=0
# dicA  list to store the first phrase
# dicB  list to store the second pharse
# val  true val




In [2]:
# remove oov
dicA.pop(254)
dicB.pop(254)
val.pop(254)


1

In [3]:
dicA.pop(254)
dicB.pop(254)
val.pop(254)

0

In [4]:
dic1 = get_dict_from_source(source1)
# add method
pred = compute_Kba(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.6385780118499013
0.6212491239992355


In [5]:
dic1 = get_dict_from_source(source2)
# add method
pred = compute_Kba(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.430334048291533
0.582089235277878


In [6]:
dic1 = get_dict_from_source(source3)
# add method
pred = compute_Kba(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.4849964959969419
0.6596232665803055


In [7]:
dic1 = get_dict_from_source(source4)
# add method
pred = compute_Kba(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.46594746119051156
0.5333942109622204


In [8]:
dic1 = get_dict_from_source(source5)
# add method
pred = compute_Kba(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.668054110301769
0.6111406060863471


In [9]:
dic1 = get_dict_from_source(source6)
# add method
pred = compute_Kba(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.7253711057784197
0.7056849795068912


In [4]:
from sklearn.metrics import roc_auc_score
dic1 = get_dict_from_source(source7)
# add method
pred = compute_Kba(dicA, dicB,dic1)

result = roc_auc_score(val, pred)


In [5]:
result


0.7283441992822103

In [6]:
import pandas as pd
dic= {'dicA':dicA,'dicB':dicB,'label':val,'pred':pred}
df = pd.DataFrame(dic)

In [7]:
pred2=[i>0 for i in pred]

In [8]:
pred2 =[int(i==True) for i in pred2]
df['pred2']=pred2

In [9]:
df

,dicA,dicB,label,pred,pred2
0,develop skill,create ability,1,0.081403,1
1,create ability,develop skill,0,-0.081403,0
2,solve problem,understand difficulty,1,-0.118591,0
3,understand difficulty,solve problem,0,0.118591,1
4,sign contract,write agreement,1,-0.036990,0
...,...,...,...,...,...
429,affect thinking,stimulate think,0,-0.026663,0
430,market film,trade product,1,0.112178,1
431,trade product,market film,0,-0.112178,0
432,advise firm,talk business,1,-0.022931,0


In [10]:
wrong=[]
for i in range(len(df)):
    if df.label[i]!=df.pred2[i]:
        wrong.append(i)

In [11]:
wrong

[2,
 3,
 4,
 5,
 6,
 7,
 24,
 25,
 26,
 27,
 36,
 37,
 44,
 45,
 48,
 49,
 54,
 55,
 56,
 57,
 64,
 65,
 68,
 69,
 70,
 71,
 72,
 73,
 80,
 81,
 86,
 87,
 90,
 91,
 92,
 93,
 96,
 97,
 106,
 107,
 116,
 117,
 132,
 133,
 136,
 137,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 154,
 155,
 158,
 159,
 164,
 165,
 166,
 167,
 174,
 175,
 184,
 185,
 198,
 199,
 200,
 201,
 206,
 207,
 210,
 211,
 232,
 233,
 234,
 235,
 236,
 237,
 278,
 279,
 284,
 285,
 286,
 287,
 300,
 301,
 312,
 313,
 316,
 317,
 324,
 325,
 326,
 327,
 328,
 329,
 332,
 333,
 334,
 335,
 346,
 347,
 348,
 349,
 352,
 353,
 362,
 363,
 372,
 373,
 378,
 379,
 380,
 381,
 384,
 385,
 390,
 391,
 396,
 397,
 402,
 403,
 404,
 405,
 414,
 415,
 420,
 421,
 424,
 425,
 426,
 427,
 432,
 433]

In [12]:
wrong =[i for i in wrong if i%2==0]

In [13]:
wrong

[2,
 4,
 6,
 24,
 26,
 36,
 44,
 48,
 54,
 56,
 64,
 68,
 70,
 72,
 80,
 86,
 90,
 92,
 96,
 106,
 116,
 132,
 136,
 144,
 146,
 148,
 150,
 154,
 158,
 164,
 166,
 174,
 184,
 198,
 200,
 206,
 210,
 232,
 234,
 236,
 278,
 284,
 286,
 300,
 312,
 316,
 324,
 326,
 328,
 332,
 334,
 346,
 348,
 352,
 362,
 372,
 378,
 380,
 384,
 390,
 396,
 402,
 404,
 414,
 420,
 424,
 426,
 432]

In [14]:
dic1 = get_dict_from_source(source8)
# add method
pred = compute_Kba(dicA, dicB,dic1)
result = roc_auc_score(val, pred)
print(result)
# pointwise


0.6995476650597804


In [15]:
dic= {'dicA':dicA,'dicB':dicB,'label':val,'pred':pred}
df2 = pd.DataFrame(dic)
pred2=[i>0 for i in pred]
pred2 =[int(i==True) for i in pred2]
df2['pred2']=pred2

In [16]:
wrong2=[]
for i in range(len(df2)):
    if df.label[i]!=df.pred2[i]:
        wrong2.append(i)

In [17]:
common=[]
for i in wrong2:
    if i in wrong:
        common.append(i)

In [19]:
df_error=df.iloc[common][:]

In [22]:
df_error[['dicA','dicB']][:12]

,dicA,dicB
2,solve problem,understand difficulty
4,sign contract,write agreement
6,reduce number,decrease amount
24,undertake research,initiate investigation
26,afford fee,pay cost
36,mark end,connect location
44,present work,show activity
48,read text,understand matter
54,conduct survey,manage investigation
56,reflect view,indicate attitude


In [12]:
dic1 = get_dict_from_source(source9)
# add method
pred = compute_Kba(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)
# pointwise
pred = compute_Kba(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.49822676208881056
0.4623372762216229


In [5]:
from sklearn.metrics import roc_auc_score
dic1 = get_dict_from_source(source1)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.6224383614007517


In [6]:
dic1 = get_dict_from_source(source2)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.6273864384463465


In [7]:
dic1 = get_dict_from_source(source3)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.6283420756439932


In [8]:
dic1 = get_dict_from_source(source4)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.5229459109346132


In [9]:
dic1 = get_dict_from_source(source5)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.5922614623372763


In [10]:
dic1 = get_dict_from_source(source6)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.6959587164730617


In [11]:
dic1 = get_dict_from_source(source7)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.5668839856442056


In [12]:
dic1 = get_dict_from_source(source8)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.6049183461105566


In [13]:
dic1 = get_dict_from_source(source9)
pred = compute_Kba(dicA, dicB,dic1,method='mult')
result = roc_auc_score(val, pred)
print(result)

0.48096158338465456
